In [1]:
# import json
import ast
from functools import partial
from typing import List
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from scipy.spatial.distance import cosine, euclidean
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.options.display.max_colwidth = 150

# Импорт данных

In [3]:
dataset = pd.read_parquet("train_pairs.parquet")
etl = pd.read_parquet("train_data.parquet")

## Транзитивность

In [4]:
dataset_trf = dataset.merge(dataset, how='inner', left_on='variantid2', right_on='variantid1')

In [5]:
# не совпадающие метки после соединения
dataset_new = dataset_trf[dataset_trf['target_x'] != dataset_trf['target_y']]
dataset_new_rtm = dataset_new[['target_y', 'variantid1_x', 'variantid2_y']]
dataset_new_rtm.columns = ['target', 'variantid1', 'variantid2']

dataset_new.head()

,target_x,variantid1_x,variantid2_x,target_y,variantid1_y,variantid2_y
14,1.0,89540591,732942986,0.0,732942986,797685052
31,1.0,91603949,485333946,0.0,485333946,523588747
37,0.0,91606616,91606667,1.0,91606667,607074418
38,1.0,96009344,229747040,0.0,229747040,418054843
50,1.0,147125215,158698192,0.0,158698192,452757806


In [6]:
# совпадающие метки после соединения
dataset_new2 = dataset_trf[dataset_trf['target_x'] == dataset_trf['target_y']]
dataset_new2_rtm = dataset_new2[['target_x', 'variantid1_x', 'variantid2_y']]
dataset_new2_rtm.columns = ['target', 'variantid1', 'variantid2']
dataset_new.head()

,target_x,variantid1_x,variantid2_x,target_y,variantid1_y,variantid2_y
14,1.0,89540591,732942986,0.0,732942986,797685052
31,1.0,91603949,485333946,0.0,485333946,523588747
37,0.0,91606616,91606667,1.0,91606667,607074418
38,1.0,96009344,229747040,0.0,229747040,418054843
50,1.0,147125215,158698192,0.0,158698192,452757806


In [7]:
# новых данных:
print(((len(dataset_new_rtm) + len(dataset_new2_rtm) + len(dataset)) / len(dataset)) - 1)
print((len(dataset_new_rtm) + len(dataset_new2_rtm) + len(dataset)) - len(dataset))

0.3498727735368956
107250


In [8]:
# Объединение исходного и новых датафреймов.
dataset = pd.concat([dataset, dataset_new_rtm, dataset_new2_rtm], axis=0)

In [9]:
dataset.shape

(413790, 3)

In [10]:
# Пары + признаки
features = (
    dataset
    .merge(
        etl
        .add_suffix('1'),
        on="variantid1"
    )
    .merge(
        etl
        .add_suffix('2'),
        on="variantid2"
    )
)

In [11]:
def compare_dicts(row):
    """
    Функция псравнивает 2 строки-словаря друг с другом.
    """
    return row['categories1'] == row['categories2']


features['categories1'] = features['categories1'].str.lower()
features['categories2'] = features['categories2'].str.lower()
features['comparison_result'] = features.apply(compare_dicts, axis=1) * 1

In [12]:
cat_missmatch = features[(features['target'] == 1) & (features['comparison_result'] == 0)]

In [13]:
def miss_match_info(cat_missmatch):
    """
    Функция берет на вход дф со строками словарями категорий.
    Возвращает новый дф с тремя столбцами:
    missmatch_cat - 1й уровень категории где есть различие в паре. Если расхождение было в категориях 2,3,4 то вернется 2.
    miss_match_obj - словарь, где ключ - категория с мисметчем по паре, значение - кортеж с названиями категорий
    missmatch_cat_cnt - в скольких уровнях был мисметч. Если расхождение было в категориях 2,3,4 то вернется 3.
    """

    miss_match_series = pd.Series(index=cat_missmatch.index, dtype='object')
    miss_match_category = pd.Series(index=cat_missmatch.index, dtype='int8')
    miss_match_count_category = pd.Series(index=cat_missmatch.index, dtype='int8')

    for i in cat_missmatch.index:
        cat1_dict = ast.literal_eval(cat_missmatch.loc[i,'categories1'])
        cat2_dict = ast.literal_eval(cat_missmatch.loc[i,'categories2'])
        if len(cat1_dict) == len(cat2_dict):
            miss_match_dict = {}
            cat_list = []
            for key in cat1_dict.keys():

                if cat1_dict[key] != cat2_dict[key]:
                    miss_match_dict[key] = tuple((cat1_dict[key], cat2_dict[key]))
                    miss_match_series.loc[i] = str(miss_match_dict)
                    cat_list.append(int(key))
            miss_match_category.loc[i] = min(cat_list)
            miss_match_count_category.loc[i] = len(cat_list)
            
    return pd.concat([miss_match_category, miss_match_series, miss_match_count_category], ignore_index=False, axis=1)

In [14]:
# features.isnull().sum()

In [15]:
features = pd.concat([features, miss_match_info(cat_missmatch)], axis=1)

In [16]:
features.columns = [                            'target',
                               'variantid1',
                               'variantid2',
                                    'name1',
                              'categories1',
                            'color_parsed1',
                'pic_embeddings_resnet_v11',
           'main_pic_embeddings_resnet_v11',
                            'name_bert_641',
       'characteristic_attributes_mapping1',
                                    'name2',
                              'categories2',
                            'color_parsed2',
                'pic_embeddings_resnet_v12',
           'main_pic_embeddings_resnet_v12',
                            'name_bert_642',
       'characteristic_attributes_mapping2',
                        'comparison_result',
                      'missmatch_cat',
                        'miss_match_obj',
                         'missmatch_cat_cnt']

In [17]:
features = features[features['missmatch_cat'] != 2]

## Парсинг категорий.

In [18]:
features.reset_index(drop=True, inplace=True)

In [19]:
features.columns

Index(['target', 'variantid1', 'variantid2', 'name1', 'categories1',
       'color_parsed1', 'pic_embeddings_resnet_v11',
       'main_pic_embeddings_resnet_v11', 'name_bert_641',
       'characteristic_attributes_mapping1', 'name2', 'categories2',
       'color_parsed2', 'pic_embeddings_resnet_v12',
       'main_pic_embeddings_resnet_v12', 'name_bert_642',
       'characteristic_attributes_mapping2', 'comparison_result',
       'missmatch_cat', 'miss_match_obj', 'missmatch_cat_cnt'],
      dtype='object')

In [20]:
nans = [np.nan for i in range(len(features))]
df_cat1 = pd.DataFrame({1:nans, 2:nans, 3:nans, 4:nans})
df_cat2 = pd.DataFrame({1:nans, 2:nans, 3:nans, 4:nans})

In [21]:
for i in tqdm_notebook(range(len(features))):    
    cat1 = ast.literal_eval(features.loc[i,'categories1'])
    cat2 = ast.literal_eval(features.loc[i,'categories2'])

    for j in range(1,5):  
        df_cat1.loc[i,j] = cat1[str(j)]
        df_cat2.loc[i,j] = cat2[str(j)] 

  0%|          | 0/413564 [00:00<?, ?it/s]

In [22]:
df_cat1.columns = ['cat1_1', 'cat2_1', 'cat3_1', 'cat4_1']
df_cat2.columns = ['cat1_2', 'cat2_2', 'cat3_2', 'cat4_2']

In [23]:
df_cat = pd.concat([df_cat1, df_cat2], axis= 1)

In [24]:
print(f'длина исходного дф {len(df_cat)}')
print(f"совпадений по категории 1го уровня - {len(df_cat[df_cat['cat1_1'] == df_cat['cat1_2']])}")
print(f"совпадений по категории 2го уровня - {len(df_cat[df_cat['cat2_1'] == df_cat['cat2_2']])}")
print(f"совпадений по категории 3го уровня - {len(df_cat[df_cat['cat3_1'] == df_cat['cat3_2']])}")
print(f"совпадений по категории 4го уровня - {len(df_cat[df_cat['cat4_1'] == df_cat['cat4_2']])}")

длина исходного дф 413564
совпадений по категории 1го уровня - 413564
совпадений по категории 2го уровня - 413541
совпадений по категории 3го уровня - 408057
совпадений по категории 4го уровня - 400591


* категория 1го уровня совпадает на всех парах независимо от таргета, ее можно исключить/

In [25]:
df_cat.isnull().sum()

cat1_1    0
cat2_1    0
cat3_1    0
cat4_1    0
cat1_2    0
cat2_2    0
cat3_2    0
cat4_2    0
dtype: int64

In [26]:
features = pd.concat([features, df_cat], axis=1)

In [27]:
features.drop(['cat1_1', 'cat1_2', 'categories1', 'categories2'], axis=1, inplace=True)

In [28]:
features.columns

Index(['target', 'variantid1', 'variantid2', 'name1', 'color_parsed1',
       'pic_embeddings_resnet_v11', 'main_pic_embeddings_resnet_v11',
       'name_bert_641', 'characteristic_attributes_mapping1', 'name2',
       'color_parsed2', 'pic_embeddings_resnet_v12',
       'main_pic_embeddings_resnet_v12', 'name_bert_642',
       'characteristic_attributes_mapping2', 'comparison_result',
       'missmatch_cat', 'miss_match_obj', 'missmatch_cat_cnt', 'cat2_1',
       'cat3_1', 'cat4_1', 'cat2_2', 'cat3_2', 'cat4_2'],
      dtype='object')

## Работа с null

In [29]:
features = features[~((features['characteristic_attributes_mapping1'].isnull()) | (features['characteristic_attributes_mapping2'].isnull()))]

In [30]:
features

,target,variantid1,variantid2,name1,color_parsed1,pic_embeddings_resnet_v11,main_pic_embeddings_resnet_v11,name_bert_641,characteristic_attributes_mapping1,name2,...,comparison_result,missmatch_cat,miss_match_obj,missmatch_cat_cnt,cat2_1,cat3_1,cat4_1,cat2_2,cat3_2,cat4_2
0,0.0,51197862,51198054,Удлинитель TDM Electric Люкс УЛ05В 5 м (SQ1303-0138),[белый],None,"[[-0.4304909, -0.49474272, -0.46439183, -0.06097448, -1.8553698, 0.9934219, 0.38884374, 0.009462222, 0.19081306, -1.2679458, 0.45550576, 1.3070302...","[-0.5104684, 0.56158644, 0.58873796, -0.5297187, 0.67423356, 0.531697, -0.04992381, -0.6909534, 0.39199966, -0.38739476, -0.4923953, 0.27993584, 0...","{""Число жил"":[""3""],""Макс. нагрузка, Вт"":[""3500""],""Стандарт защиты"":[""IP20""],""Номинальный ток, А"":[""16""],""Цвет товара"":[""белый""],""Электробезопаснос...",Удлинитель TDM Electric Люкс УЛ05В 1.5 м (SQ1303-0136),...,1,NaN,NaN,NaN,электроника,"сетевые фильтры, разветвители и удлинители","сетевой фильтр, удлинитель, разветвитель",электроника,"сетевые фильтры, разветвители и удлинители","сетевой фильтр, удлинитель, разветвитель"
1,0.0,51197862,51199884,Удлинитель TDM Electric Люкс УЛ05В 5 м (SQ1303-0138),[белый],None,"[[-0.4304909, -0.49474272, -0.46439183, -0.06097448, -1.8553698, 0.9934219, 0.38884374, 0.009462222, 0.19081306, -1.2679458, 0.45550576, 1.3070302...","[-0.5104684, 0.56158644, 0.58873796, -0.5297187, 0.67423356, 0.531697, -0.04992381, -0.6909534, 0.39199966, -0.38739476, -0.4923953, 0.27993584, 0...","{""Число жил"":[""3""],""Макс. нагрузка, Вт"":[""3500""],""Стандарт защиты"":[""IP20""],""Номинальный ток, А"":[""16""],""Цвет товара"":[""белый""],""Электробезопаснос...",Удлинитель TDM Electric Люкс УЛ05В 3 м (SQ1303-0137),...,1,NaN,NaN,NaN,электроника,"сетевые фильтры, разветвители и удлинители","сетевой фильтр, удлинитель, разветвитель",электроника,"сетевые фильтры, разветвители и удлинители","сетевой фильтр, удлинитель, разветвитель"
2,0.0,51197862,51199877,Удлинитель TDM Electric Люкс УЛ05В 5 м (SQ1303-0138),[белый],None,"[[-0.4304909, -0.49474272, -0.46439183, -0.06097448, -1.8553698, 0.9934219, 0.38884374, 0.009462222, 0.19081306, -1.2679458, 0.45550576, 1.3070302...","[-0.5104684, 0.56158644, 0.58873796, -0.5297187, 0.67423356, 0.531697, -0.04992381, -0.6909534, 0.39199966, -0.38739476, -0.4923953, 0.27993584, 0...","{""Число жил"":[""3""],""Макс. нагрузка, Вт"":[""3500""],""Стандарт защиты"":[""IP20""],""Номинальный ток, А"":[""16""],""Цвет товара"":[""белый""],""Электробезопаснос...",Удлинитель TDM Electric Люкс УЛ05В 5 м (SQ1303-0138),...,1,NaN,NaN,NaN,электроника,"сетевые фильтры, разветвители и удлинители","сетевой фильтр, удлинитель, разветвитель",электроника,"сетевые фильтры, разветвители и удлинители","сетевой фильтр, удлинитель, разветвитель"
3,0.0,51198054,51199877,Удлинитель TDM Electric Люкс УЛ05В 1.5 м (SQ1303-0136),[белый],None,"[[-0.42941108, -0.5129398, -0.4753536, -0.06778808, -1.8299942, 0.99170506, 0.403189, 0.029683203, 0.14085612, -1.2407316, 0.4877758, 1.3187227, -...","[-0.455473, 0.58157134, 0.5870387, -0.5325003, 0.6744413, 0.5440314, -0.03000803, -0.6848395, 0.37924743, -0.370057, -0.44283366, 0.24272914, 0.68...","{""Электробезопасность"":[""Заземление""],""Длина кабеля питания, м"":[""1.5""],""Входное напряжение, В"":[""250""],""Вид кабеля"":[""ПВС""],""Тип"":[""Удлинитель бы...",Удлинитель TDM Electric Люкс УЛ05В 5 м (SQ1303-0138),...,1,NaN,NaN,NaN,электроника,"сетевые фильтры, разветвители и удлинители","сетевой фильтр, удлинитель, разветвитель",электроника,"сетевые фильтры, разветвители и удлинители","сетевой фильтр, удлинитель, разветвитель"
4,1.0,53062686,536165289,Картридж лазерный Комус 729 (4370B002) черный для CanonLBP7010C/7018,"[черный, чер]",None,"[[-0.0032773763, 0.32531193, -0.33156675, 0.410158, 0.87286234, 0.29750592, -0.59826434, -0.105307996, 0.7658103, -0.8648213, 0.18156853, -0.12662...","[-0.53706163, 0.37264067, 0.44363657, -0.37289095, 0.09633542, 0.38992903, 0.10306197, -0.5976585, 0.6117

In [31]:
features.loc[features['color_parsed1'].notnull(), 'color_parsed1'] = features[features['color_parsed1'].notnull()]['color_parsed1'].apply(lambda x: list(x)[0])
features.loc[features['color_parsed2'].notnull(), 'color_parsed2'] = features[features['color_parsed2'].notnull()]['color_parsed2'].apply(lambda x: list(x)[0])

In [32]:
# группировка для определения топ цвета в разрезе категория-2,категория-3 для 1го товара
top_colors_by_cat_1 = features[features['color_parsed1'].notnull()] \
                        .groupby(['cat3_1', 'color_parsed1'], as_index=False).agg({'target':np.size})
# top_colors_by_cat_1

In [33]:
top_colors_by_cat_1[top_colors_by_cat_1['cat3_1'] == 'блоки питания'].iloc[2,1]

'gold'

In [34]:
# группировка для определения топ цвета в разрезе категория-2,категория-3 для 2го товара
top_colors_by_cat_2 = features[features['color_parsed2'].notnull()] \
                        .groupby(['cat3_2', 'color_parsed2'], as_index=False).agg({'target':np.size})
# top_colors_by_cat_2

In [35]:
features.reset_index(drop=True, inplace=True)

In [36]:
features

,target,variantid1,variantid2,name1,color_parsed1,pic_embeddings_resnet_v11,main_pic_embeddings_resnet_v11,name_bert_641,characteristic_attributes_mapping1,name2,...,comparison_result,missmatch_cat,miss_match_obj,missmatch_cat_cnt,cat2_1,cat3_1,cat4_1,cat2_2,cat3_2,cat4_2
0,0.0,51197862,51198054,Удлинитель TDM Electric Люкс УЛ05В 5 м (SQ1303-0138),белый,None,"[[-0.4304909, -0.49474272, -0.46439183, -0.06097448, -1.8553698, 0.9934219, 0.38884374, 0.009462222, 0.19081306, -1.2679458, 0.45550576, 1.3070302...","[-0.5104684, 0.56158644, 0.58873796, -0.5297187, 0.67423356, 0.531697, -0.04992381, -0.6909534, 0.39199966, -0.38739476, -0.4923953, 0.27993584, 0...","{""Число жил"":[""3""],""Макс. нагрузка, Вт"":[""3500""],""Стандарт защиты"":[""IP20""],""Номинальный ток, А"":[""16""],""Цвет товара"":[""белый""],""Электробезопаснос...",Удлинитель TDM Electric Люкс УЛ05В 1.5 м (SQ1303-0136),...,1,NaN,NaN,NaN,электроника,"сетевые фильтры, разветвители и удлинители","сетевой фильтр, удлинитель, разветвитель",электроника,"сетевые фильтры, разветвители и удлинители","сетевой фильтр, удлинитель, разветвитель"
1,0.0,51197862,51199884,Удлинитель TDM Electric Люкс УЛ05В 5 м (SQ1303-0138),белый,None,"[[-0.4304909, -0.49474272, -0.46439183, -0.06097448, -1.8553698, 0.9934219, 0.38884374, 0.009462222, 0.19081306, -1.2679458, 0.45550576, 1.3070302...","[-0.5104684, 0.56158644, 0.58873796, -0.5297187, 0.67423356, 0.531697, -0.04992381, -0.6909534, 0.39199966, -0.38739476, -0.4923953, 0.27993584, 0...","{""Число жил"":[""3""],""Макс. нагрузка, Вт"":[""3500""],""Стандарт защиты"":[""IP20""],""Номинальный ток, А"":[""16""],""Цвет товара"":[""белый""],""Электробезопаснос...",Удлинитель TDM Electric Люкс УЛ05В 3 м (SQ1303-0137),...,1,NaN,NaN,NaN,электроника,"сетевые фильтры, разветвители и удлинители","сетевой фильтр, удлинитель, разветвитель",электроника,"сетевые фильтры, разветвители и удлинители","сетевой фильтр, удлинитель, разветвитель"
2,0.0,51197862,51199877,Удлинитель TDM Electric Люкс УЛ05В 5 м (SQ1303-0138),белый,None,"[[-0.4304909, -0.49474272, -0.46439183, -0.06097448, -1.8553698, 0.9934219, 0.38884374, 0.009462222, 0.19081306, -1.2679458, 0.45550576, 1.3070302...","[-0.5104684, 0.56158644, 0.58873796, -0.5297187, 0.67423356, 0.531697, -0.04992381, -0.6909534, 0.39199966, -0.38739476, -0.4923953, 0.27993584, 0...","{""Число жил"":[""3""],""Макс. нагрузка, Вт"":[""3500""],""Стандарт защиты"":[""IP20""],""Номинальный ток, А"":[""16""],""Цвет товара"":[""белый""],""Электробезопаснос...",Удлинитель TDM Electric Люкс УЛ05В 5 м (SQ1303-0138),...,1,NaN,NaN,NaN,электроника,"сетевые фильтры, разветвители и удлинители","сетевой фильтр, удлинитель, разветвитель",электроника,"сетевые фильтры, разветвители и удлинители","сетевой фильтр, удлинитель, разветвитель"
3,0.0,51198054,51199877,Удлинитель TDM Electric Люкс УЛ05В 1.5 м (SQ1303-0136),белый,None,"[[-0.42941108, -0.5129398, -0.4753536, -0.06778808, -1.8299942, 0.99170506, 0.403189, 0.029683203, 0.14085612, -1.2407316, 0.4877758, 1.3187227, -...","[-0.455473, 0.58157134, 0.5870387, -0.5325003, 0.6744413, 0.5440314, -0.03000803, -0.6848395, 0.37924743, -0.370057, -0.44283366, 0.24272914, 0.68...","{""Электробезопасность"":[""Заземление""],""Длина кабеля питания, м"":[""1.5""],""Входное напряжение, В"":[""250""],""Вид кабеля"":[""ПВС""],""Тип"":[""Удлинитель бы...",Удлинитель TDM Electric Люкс УЛ05В 5 м (SQ1303-0138),...,1,NaN,NaN,NaN,электроника,"сетевые фильтры, разветвители и удлинители","сетевой фильтр, удлинитель, разветвитель",электроника,"сетевые фильтры, разветвители и удлинители","сетевой фильтр, удлинитель, разветвитель"
4,1.0,53062686,536165289,Картридж лазерный Комус 729 (4370B002) черный для CanonLBP7010C/7018,черный,None,"[[-0.0032773763, 0.32531193, -0.33156675, 0.410158, 0.87286234, 0.29750592, -0.59826434, -0.105307996, 0.7658103, -0.8648213, 0.18156853, -0.12662...","[-0.53706163, 0.37264067, 0.44363657, -0.37289095, 0.09633542, 0.38992903, 0.10306197, -0.5976585, 0.6117227, -0.34984562,

In [37]:
for i in tqdm_notebook(range(len(features))):
    if features.loc[i,'color_parsed1'] is None:
        try:
            features.loc[i,'color_parsed1'] = top_colors_by_cat_1[(top_colors_by_cat_1['cat3_1'] == features.loc[i, 'cat3_1'])] \
                                                            .nlargest(1, 'target')['color_parsed1']
        except:
            features.loc[i,'color_parsed1'] = 'черный'
    if features.loc[i,'color_parsed2'] is None:
        try:
            features.loc[i,'color_parsed2'] = top_colors_by_cat_2[(top_colors_by_cat_2['cat3_2'] == features.loc[i, 'cat3_2'])] \
                                                            .nlargest(1, 'target')['color_parsed2']
        except:
            features.loc[i,'color_parsed2'] = 'черный'
    

  0%|          | 0/413533 [00:00<?, ?it/s]

In [38]:
features[['color_parsed1']].value_counts()

color_parsed1  
черный             239728
белый               27995
серый               22170
black               22106
серебристый         11310
                    ...  
гранитный               1
темно-оливковый         1
перу                    1
томатный                1
amber                   1
Length: 189, dtype: int64

In [39]:
features[['color_parsed2']].value_counts()

color_parsed2
черный           239369
белый             27873
black             22719
серый             21532
серебристый       10848
                  ...  
vanilla               1
pear                  1
терракота             1
томатный              1
amber                 1
Length: 193, dtype: int64

In [40]:
features.columns

Index(['target', 'variantid1', 'variantid2', 'name1', 'color_parsed1',
       'pic_embeddings_resnet_v11', 'main_pic_embeddings_resnet_v11',
       'name_bert_641', 'characteristic_attributes_mapping1', 'name2',
       'color_parsed2', 'pic_embeddings_resnet_v12',
       'main_pic_embeddings_resnet_v12', 'name_bert_642',
       'characteristic_attributes_mapping2', 'comparison_result',
       'missmatch_cat', 'miss_match_obj', 'missmatch_cat_cnt', 'cat2_1',
       'cat3_1', 'cat4_1', 'cat2_2', 'cat3_2', 'cat4_2'],
      dtype='object')

In [42]:
features.isnull().sum()

target                                     0
variantid1                                 0
variantid2                                 0
name1                                      0
color_parsed1                              0
pic_embeddings_resnet_v11             104246
main_pic_embeddings_resnet_v11             0
name_bert_641                              0
characteristic_attributes_mapping1         0
name2                                      0
color_parsed2                              0
pic_embeddings_resnet_v12             108852
main_pic_embeddings_resnet_v12             0
name_bert_642                              0
characteristic_attributes_mapping2         0
comparison_result                          0
missmatch_cat                         405469
miss_match_obj                        405469
missmatch_cat_cnt                     405469
cat2_1                                     0
cat3_1                                     0
cat4_1                                     0
cat2_2    

In [43]:
features.drop('comparison_result', axis=1, inplace=True)